# library

In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import random

2022-06-11 06:04:12.866990: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [31]:
d1215 = pd.read_feather('d1215.ftr', columns = None, use_threads = True)

In [3]:
d1215.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1084047 entries, 0 to 1084046
Columns: 823 entries, date to NextChange
dtypes: category(1), float64(821), int64(1)
memory usage: 6.6 GB


In [32]:
d1215['code'] = d1215['code'].astype(str)

In [33]:
d1215.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1084047 entries, 0 to 1084046
Columns: 823 entries, date to NextChange
dtypes: float64(821), int64(1), object(1)
memory usage: 6.6+ GB


In [16]:
d1215['code'].unique()

array(['016385', '090120', '131090', ..., '196450', '204650', '196490'],
      dtype=object)

# data load

In [13]:
# d1221 = pd.read_csv('/data/stock/stock.KOSPI.KOSDAQ.2012to20220301.csv')
# 그냥 하나하나 불러와서 쓰기

In [14]:
# d1221.shape

(2923868, 823)

In [15]:
# sum(d1221['date']>20211231)

80818

In [16]:
2923868-80818

2843050

In [36]:
# d1221_sort[:1084047]
# d1221.sort_values(['date', 'code'])

## mk_cd 함수

In [2]:
## 함수
def mk_cd(x):
    if type(x) == str:
        if x.isalpha():
            return x
        else:
            if len(x) == 6:
                return x
            else:
                return "0"*(6-len(x))+x
    elif type(x) == int:
        x = str(x)
        return "0"*(6-len(x))+x

In [7]:
## 적용
# d1221['code'] = list(map(mk_cd, d1221['code']))

In [8]:
# d1221_sort = d1221.sort_values(['code', 'date']).reset_index(drop = True)
# d1221_sort
# d1221.iloc[np.where(d1221['date']<=20151231)]

# data load2 for concat

In [ ]:
d2016 = pd.read_csv('/data/stock/stock.KOSPI.KOSDAQ.2016.csv')
d2017 = pd.read_csv('/data/stock/stock.KOSPI.KOSDAQ.2017.csv')
d2018 = pd.read_csv('/data/stock/stock.KOSPI.KOSDAQ.2018.csv')
d2019 = pd.read_csv('/data/stock/stock.KOSPI.KOSDAQ.2019.csv')
d2020 = pd.read_csv('/data/stock/stock.KOSPI.KOSDAQ.2020.csv')
d2021 = pd.read_csv('/data/stock/stock.KOSPI.KOSDAQ.2021.csv')

In [ ]:
# modeling시 error 방지 위해 NaN값 채우고 ftr 만들기...
d2016 = d2016.fillna(d2016.iloc[:,2:].mean())
d2018 = d2018.fillna(d2018.iloc[:,2:].mean())
d2020 = d2020.fillna(d2020.iloc[:,2:].mean())

In [34]:
d1221 = pd.concat([d1215, d2016, d2017, d2018, d2019, d2020, d2021])

d1221.shape
# 2843050 맞음

(2843050, 823)

## d1221 to ftr

In [35]:
d1221.reset_index(drop=True, inplace = True)

In [36]:
d1221['code'] = list(map(mk_cd, d1221['code'])) # mk_cd함수 적용시켜야 ftr만들 때 에러 안남.

In [37]:
d1221['code']=d1221['code'].astype('category')

In [38]:
d1221.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2843050 entries, 0 to 2843049
Columns: 823 entries, date to NextChange
dtypes: category(1), float64(821), int64(1)
memory usage: 17.4 GB


In [39]:
# d1221.to_feather('d1221.ftr')

# data load 3 여기부터 하면 됨.

In [3]:
len_a2012 = 284412
len_a2013 = 242826
len_a2014 = 242448
len_a2015 = 314361
len_a2016 = 263593
len_a2017 = 216591
len_a2018 = 307395
len_a2019 = 278610
len_a2020 = 371006
len_a2021 = 321808

# 슬라이싱 테스트

In [4]:
len1215 = len_a2012+len_a2013+len_a2014+len_a2015 # y:2015
len1316 = len1215+len_a2016 # y:2016
len1417 = len1316+len_a2017 # y:2017
len1518 = len1417+len_a2018 # y:2018
len1619 = len1518+len_a2019 # y:2019
len1720 = len1619+len_a2020 # y:2020
len1821 = len1720+len_a2021 # y:2021

# [0], [1]은 train 슬라이싱 인덱스, [2]는 test인덱스로 구성

In [5]:
slicing_dic = {'1215':[0, len1215, len_a2015],
               '1316':[len_a2012, len1316, len_a2016],
               '1417':[len_a2012+len_a2013, len1417, len_a2017],
               '1518':[len_a2012+len_a2013+len_a2014, len1518, len_a2018],
               '1619':[len_a2012+len_a2013+len_a2014+len_a2015, len1619, len_a2019],
               '1720':[len_a2012+len_a2013+len_a2014+len_a2015+len_a2016,len1720, len_a2020],
               '1821':[len_a2012+len_a2013+len_a2014+len_a2015+len_a2016+len_a2017,len1821, len_a2021]}

In [6]:
# d1221[:len1215]
# d1221[len_a2012:len1316]
# d1221[len_a2012+len_a2013:len1417]
# d1221[len_a2012+len_a2013+len_a2014:len1518]
# d1221[len_a2012+len_a2013+len_a2014+len_a2015:len1619]
# d1221[len_a2012+len_a2013+len_a2014+len_a2015+len_a2016:len1720]
# d1221[len_a2012+len_a2013+len_a2014+len_a2015+len_a2016+len_a2017:len1821]

In [6]:
d1221 = pd.read_feather('d1221.ftr', columns = None, use_threads = True)

In [7]:
d1221.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2843050 entries, 0 to 2843049
Columns: 823 entries, date to NextChange
dtypes: category(1), float64(821), int64(1)
memory usage: 17.4 GB


# category to object

In [8]:
d1221['code'] = list(map(mk_cd, d1221['code'])) # mk_cd함수 적용시켜야 ftr만들 때 에러 안남.

In [9]:
d1221.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2843050 entries, 0 to 2843049
Columns: 823 entries, date to NextChange
dtypes: float64(821), int64(1), object(1)
memory usage: 17.4+ GB


In [11]:
# train test split test cell
# X = d1221.iloc[:,2:-1] # 1221 데이터
# y = d1221.iloc[:,-1]
for enu, i in enumerate(slicing_dic):
    slicing_dic[i][0],slicing_dic[i][1]
    tmp_all = d1221[slicing_dic[i][0]:slicing_dic[i][1]].reset_index(drop = True) # 전체데이터
    if enu>=1:
        i, slicing_dic[i]
        tmp_train = tmp_all[:-slicing_dic[i][2]]
        tmp_test = tmp_all[-slicing_dic[i][2]:]
        
#         slicing_dic[i][1]-slicing_dic[i][0] # all과 결과 같아야함. #같음
        'all',tmp_all.shape[0]
        
#         slicing_dic[i][1]-(slicing_dic[i][2]+slicing_dic[i][0]) # tr과 결과 같음
        'tr',tmp_train.shape[0]
#         tmp_train
        'te',tmp_test.shape[0]
#         tmp_test
        # train끼리 idx 같아야 하므로
        len_train = slicing_dic[i][1]-(slicing_dic[i][2]+slicing_dic[i][0]) # 혹은 tmp_train.shape[0]
        len(list(range(len_train)))
        train_idx = list(range(len_train))
        random.shuffle(train_idx) # 자동으로 덮어쓰기

        len_train_to_test = slicing_dic[i][1]-slicing_dic[i][0]
        test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
        len(test_idx)
        random.shuffle(test_idx) # 자동으로 덮어쓰기

        x_train = tmp_train.loc[train_idx]
#         y_train = tmpy.iloc[train_idx]
        x_test = tmp_test.loc[test_idx]
#         y_test = tmpy.iloc[test_idx]
        x_train
        x_test
        break

(0, 1084047)

(284412, 1347640)

('1316', [284412, 1347640, 263593])

('all', 1063228)

('tr', 799635)

('te', 263593)

799635

263593

,date,code,D-10_open,D-10_high,D-10_low,D-10_close,D-10_MA5,D-10_MA20,D-10_MA60,D-10_MA120,...,D-1_VIpos,D-1_ROC,D-1_MFI,D-1_KCW,D-1_BBP,D-1_KST,D-1_DCW,D-1_SRSID,D-1_PPOhist,NextChange
397928,20141111,094940,0.007557,0.032242,0.007557,0.029723,-0.005139,0.039144,0.032343,0.113249,...,1.06060,7.5560,37.059,7.1012,0.59287,8.0378,7.2768,0.831900,0.159820,-0.24
347410,20140108,052270,0.013554,0.022590,0.000000,0.000000,-0.000301,0.197666,0.324518,0.380377,...,0.73558,-12.7030,23.721,9.9397,0.15670,-250.9800,14.6330,0.304010,-0.701580,1.69
459109,20140305,056360,-0.001302,0.007812,-0.024740,0.002604,0.019271,0.022344,-0.066380,-0.074818,...,1.17420,4.6541,72.994,7.0364,0.86086,61.0270,13.2350,0.957520,0.473800,2.64
565355,20150907,140910,0.009823,0.009823,-0.021562,0.009823,0.010541,0.046215,0.064590,0.116842,...,1.02380,5.5991,70.885,9.6143,0.63884,-17.6620,15.6040,0.838580,0.575760,0.00
346801,20140630,003100,0.005540,0.005540,-0.016620,-0.016620,-0.003878,-0.005374,0.021939,0.001496,...,1.19260,10.8640,78.666,4.9959,1.22210,21.6230,11.8380,0.782710,0.997120,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513373,20151020,033290,0.000000,0.035032,-0.006369,0.017516,0.021656,0.006688,0.006306,-0.168025,...,0.83412,-5.7187,29.634,10.2950,0.26396,15.4410,10.2670,0.276890,-0.604720,0.00
564497,20150722,083550,-0.005558,0.013254,-0.046601,0.013254,-0.005173,0.085785,0.017914,-0.104425,...,0.95995,2.6693,55.011,11.0730,0.40023,-94.9020,13.4330,0.473430,0.043338,3.88
541804,20150519,001130,0.000000,0.000000,-0.037975,-0.012658,0.012658,0.038101,-0.038400,-0.101671,...,1.00390,14.6700,70.248,7.8521,1.40850,35.5280,21.5650,0.789560,0.800960,-4.05
680109,20150716,086520,0.000000,0.077254,-0.029063,-0.019425,-0.051423,-0.107043,-0.148918,-0.166237,...,0.93442,-5.3801,54.132,10.7000,0.14475,103.9100,21.7210,0.075171,-1.161500,-0.55


,date,code,D-10_open,D-10_high,D-10_low,D-10_close,D-10_MA5,D-10_MA20,D-10_MA60,D-10_MA120,...,D-1_VIpos,D-1_ROC,D-1_MFI,D-1_KCW,D-1_BBP,D-1_KST,D-1_DCW,D-1_SRSID,D-1_PPOhist,NextChange
864915,20161111,124500,0.000000,0.023901,-0.100939,-0.036705,0.004055,0.075822,0.216539,0.160478,...,0.80723,-11.5460,19.228,11.6110,0.187440,-145.290,29.7480,0.276430,-0.75549,0.31
868088,20160219,108230,0.000000,0.025931,-0.014850,0.025931,-0.014473,-0.065280,0.068823,0.058043,...,1.16380,7.0725,71.661,8.3316,0.928660,55.227,14.6770,0.742970,1.18560,-1.74
940832,20160107,053110,0.016277,0.016277,-0.030519,-0.024415,0.009156,0.050051,-0.043795,-0.061211,...,0.75231,-11.6400,21.798,10.4130,0.029671,-56.380,13.7710,0.095106,-1.43160,1.95
981490,20160808,111820,-0.004796,0.023979,-0.004796,0.009592,0.000026,0.007466,0.035036,0.105418,...,1.05710,9.1873,70.538,10.0260,0.766510,57.114,21.0850,0.392540,0.46186,8.41
1055788,20161005,114450,0.000000,0.011450,-0.017176,0.000000,-0.021374,0.022137,0.178244,0.212080,...,1.07380,2.7723,57.365,6.5471,0.370660,-78.095,7.5089,0.704380,0.56566,1.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899397,20161212,002785,-0.006298,0.013777,-0.050187,-0.050187,0.005235,0.011179,-0.021649,-0.119917,...,0.90029,-9.3511,63.884,11.2870,0.349690,-34.042,14.3760,0.051159,-0.64969,2.41
800754,20160829,065560,0.001456,0.001456,-0.026504,-0.004417,-0.004709,0.045386,0.028300,0.003301,...,0.87835,-5.9851,35.958,8.7871,-0.037540,-71.213,12.8090,0.024615,-0.69595,-1.91
912232,20161027,011300,0.000000,0.031963,-0.022831,-0.013699,0.025571,0.021005,-0.015616,-0.040000,...,0.84076,-8.0508,36.043,8.2547,0.488110,-18.376,13.1300,0.071234,-0.51794,-4.61
805735,20160224,024720,-0.047855,-0.021452,-0.051155,-0.049505,-0.003300,-0.002558,0.006056,0.053432,...,0.59549,-29.7390,19.918,9.7627,0.176240,-211.730,43.0450,0.091073,-2.59820,4.07


In [12]:
results = []
def my_regressor():
    X = d1221.iloc[:,2:-1] # 1221 데이터
    y = d1221.iloc[:,-1]
    from sklearn.linear_model import LinearRegression # 일반 회귀 모델
    from sklearn.linear_model import Ridge, Lasso, ElasticNet # Norm 규제 회귀 모델
    from sklearn.linear_model import ARDRegression, BayesianRidge # 베이지안 회귀
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor # decision tree 앙상블 모델, 배깅
    from xgboost import XGBRegressor # decision tree 앙상블 모델, 부스팅
    import lightgbm as lgb
    from sklearn.model_selection import GridSearchCV
#     from sklearn.model_selection import cross_val_score # model 검증
    
#     from sklearn.model_selection import StratifiedShuffleSplit
    from sklearn.model_selection import ShuffleSplit
    from sklearn.model_selection import cross_val_score
    
    from sklearn.dummy import DummyRegressor
    from sklearn.metrics import mean_squared_error as mse # 모델 평가 지표 scoring (mse)
    from sklearn.metrics import r2_score as r2
    
    import ast # convert string to function
    
    # seed 고정
    user_seed = 0
    random.seed(user_seed) # seed 고정
    
    # (같은 줄에 써놓은 애들끼리 하이퍼파리미터 구성 비슷하거나 같음)
    linear = LinearRegression(n_jobs=16)
    ridge, lasso, elasticnet = Ridge(), Lasso(alpha=0.01, max_iter=10, random_state=user_seed), ElasticNet(alpha=0.01, max_iter=10, random_state=user_seed)
    # [lasso 학습시간 단축 하이퍼파라미터 참고](https://tensorflow.blog/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/2-3-3-%EC%84%A0%ED%98%95-%EB%AA%A8%EB%8D%B8/comment-page-1/)
    # [elastic net 참고](https://soobarkbar.tistory.com/30)
    # elastic net은 큰 데이터 셋에서 잘 작동함. + ridge&lasso의 장점 모두 가짐.
    # [모델훈련](https://taek98.tistory.com/15)
    ardr_linear, bayesian_ridge = ARDRegression(verbose = True), BayesianRidge(verbose = True) # n_iter=10, 
    DecisionTree = DecisionTreeRegressor(max_depth = 10, random_state=user_seed)
    random_forest = RandomForestRegressor(n_estimators=10,
                                          n_jobs=16,
                                          max_depth=10,
                                          random_state=user_seed,
                                          verbose=2,
                                          max_leaf_nodes=4)
    xgboost_linear = XGBRegressor(nthread=16)
    gbm = lgb.LGBMRegressor(num_leaves=31,
                            learning_rate=0.05,
                            n_estimators=20)

    dummy = DummyRegressor(strategy = 'mean')
    
    my_model_list = ['linear', 'ridge', 'lasso', 'elasticnet',
                     'ardr_linear', 'bayesian_ridge',
                     'xgboost_linear', 'gbm', 'dummy',
                     'DecisionTree', 'random_forest'] # 오래 걸리는건 뒤로 뺌
    
    i = 0
    ######################################################################### fitting
    for model_nm in tqdm(my_model_list):
        # cv
        for slicer in slicing_dic:
            tmpX = X[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True)
            tmpy = y[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True)
#             tmp_all = d1221[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # 전체데이터

            X_train = tmpX[:-slicing_dic[slicer][2]]
            X_test = tmpX[-slicing_dic[slicer][2]:]
            y_train = tmpy[:-slicing_dic[slicer][2]]
            y_test = tmpy[-slicing_dic[slicer][2]:]
            
            # train끼리 idx 같아야 하므로
            len_train = slicing_dic[slicer][1]-(slicing_dic[slicer][2]+slicing_dic[slicer][0]) # 혹은 tmp_train.shape[0]
            len(list(range(len_train)))
            train_idx = list(range(len_train))
            random.shuffle(train_idx) # 자동으로 덮어쓰기

            len_train_to_test = slicing_dic[slicer][1]-slicing_dic[slicer][0]
            test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
            len(test_idx)
            random.shuffle(test_idx) # 자동으로 덮어쓰기
            
            X_train = X_train.loc[train_idx]
            y_train = y_train.loc[train_idx]
            X_test = X_test.loc[test_idx]
            y_test = y_test.loc[test_idx]
            ###########################################
            y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
            y_test = np.array(y_test).reshape(-1,1)

            ######################################################## cv를 위한 데이터 slicing 작업 끝

            # fitting은 cv 안에 넣기
            if model_nm == 'gbm':
                model_ = eval(model_nm).fit(X_train, y_train,
                                            eval_set=[(X_test, y_test)],
                                            eval_metric='l1',
                                            early_stopping_rounds=5)
                y_pred_test = model_.predict(X_test, num_iteration=gbm.best_iteration_) # 예측

            else:
                model_ = eval(model_nm).fit(X_train, y_train)
                y_pred_test = model_.predict(X_test) # 예측

            mse_score = mse(y_test, y_pred_test)
            r2_score = r2(y_test, y_pred_test)
            cv_scores = cross_val_score(eval(model_nm), tmpX, tmpy, cv=7)

            results.append([i, f'{model_nm}', mse_score, r2_score, cv_scores]) #[:str(model).index("(")]
            i+=1
        ################################################################
        print(results[-7:])
    return results#score_dic#[0]

[scikit-learn page](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html?highlight=bayesian#sklearn.linear_model.BayesianRidge)

In [13]:
import time
start = time.time()
_ = my_regressor()
results
time.time()-start
# lasso, elasocnet 하이퍼 파라미터 튜닝해주니 빨라짐. (max_iter=10으로 튜닝)
# (ardr_linear, bayesian_ridge의 n_iter는 안하고 걍 vervose만 켬, 그냥 자체로 빠름)
# xgbregressor nthread=16으로 튜닝해줌.
# linear_regression 잘돼다 갑자기 안돼서 n_jobs =16으로 튜닝함
# 순서대로 학습에
# linear: 2분, (?? 9분 시작~)
# ridge: 1분,
# lasso: 1분(튜닝후),
# elasicnet: 1분(튜닝후),
# ardr_linear: 1분,
# bayesian_ridge: 3분,
# xgbregressor: 

  0%|                                                    | 0/11 [09:46<?, ?it/s]


KeyboardInterrupt: 

# plot을 위한 준비

In [ ]:
dic = {}
for i in results:
    if i[1] not in dic: # dic.keys()라고 써도 됨.
        dic[i[1]] = [i[2]]
    else:
        dic[i[1]].append(i[2])


# import math
# 리스트에 mean 함수가 내장이 안돼있다니...

for i in dic.keys():
    dic[i] = sum(dic[i])/len(dic[i])


score_lst = sorted(dic.items(), key = lambda t : t[1]) #mse 작은 순으로 줄 세워짐.
score_lst
# dic.keys()

In [ ]:
score_dic = {}
for i in score_lst:
    score_dic[i[0]] = i[1]

In [ ]:
df = pd.DataFrame(score_dic.items(), columns=['key', 'value'])
df

In [ ]:
df[df.key=='dummy']['value'].values
f'dummy\n: {dummy_mean[0]:.2f}'

# MSE plot

In [ ]:
g = sns.barplot(x = 'key', y = 'value', data=df, capsize=.2)
_ = g.set_xticks(range(len(df)))
_ = g.set_xticklabels(g.get_xticklabels(), rotation=45)
_ = g.set_xlabel("regression models", fontsize = 15)
_ = g.set_ylabel("Mean Squared Errors", fontsize = 15)
_ = g.set_ylim([0, 0.0155]) # ax.set(ylim=(0.4, 0.6))
dummy_mean = df[df.key=='dummy']['value'].values
_ = g.plot([-1,len(df)], [dummy_mean, dummy_mean], color = 'red')
_ = g.annotate(text = f'dummy\n: {dummy_mean[0]:.4f}', xy = (3, dummy_mean), xytext = (0.1, 0.0115),
               arrowprops = dict(facecolor = 'red', shrink = 0),)


val = df['value']
cnt = 0
for i in range(len(df)):
    _ = g.text(cnt-0.27, 0.0007, f'{val[i]:.2f}')
    cnt += 1

plt.tight_layout()